Ethan Landers | CS 625 | April 11th, 2024

Old Dominion University

# Semester Project

In [2]:
import pandas as pd
import altair as alt

In [3]:
# Read in dataset
data = pd.read_csv("https://raw.githubusercontent.com/odu-cs625-datavis/Spring24-asv-ethanlanders/main/semester_project/all-weeks-global.csv?token=GHSAT0AAAAAACQLUCY7OMGO7U6OKJQNLTXSZRC2FIQ")

# View dataset
data

,week,category,weekly_rank,show_title,season_title,weekly_hours_viewed,runtime,weekly_views,cumulative_weeks_in_top_10,is_staggered_launch,episode_launch_details
0,2024-03-31,Films (English),1,Heart of the Hunter,NaN,19700000,1.7833,11000000.0,1,False,NaN
1,2024-03-31,Films (English),2,Irish Wish,NaN,16400000,1.5667,10500000.0,3,False,NaN
2,2024-03-31,Films (English),3,Damsel,NaN,18900000,1.8333,10300000.0,4,False,NaN
3,2024-03-31,Films (English),4,The Accountant,NaN,13200000,2.1333,6200000.0,1,False,NaN
4,2024-03-31,Films (English),5,The Casagrandes Movie,NaN,6100000,1.4167,4300000.0,2,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5755,2021-07-04,TV (Non-English),6,Elite,Elite: Season 1,10530000,NaN,NaN,1,False,NaN
5756,2021-07-04,TV (Non-English),7,Elite,Elite: Season 3,10200000,NaN,NaN,1,False,NaN
5757,2021-07-04,TV (Non-English),8,Elite,Elite: Season 2,10140000,NaN,NaN,1,False,NaN
5758,2021-07-04,TV (Non-English),9,Katla,Katla: Season 1,9190000,NaN,NaN,1,False,NaN


## Question 1

**Does the runtime of Netflix films influence their weekly viewership?**


In [4]:
# Define a Data Frame containing only English film information
films = data[(data['category']=="Films (English)") | (data['category']=="Films (Non-English)")]

# Remove rows from Data Frame that do not contain runtime information
films_cleaned = films.dropna(subset=['runtime'])

# Group by 'show_title' to avoid redundancy
films_grouped = films_cleaned.groupby('show_title').agg({
    'week': 'count',
    'weekly_views': 'mean',
    'runtime': 'mean'
}).reset_index()

# Plot
scatterplot = alt.Chart(films_grouped).mark_circle(size=50).encode(
    x=alt.X('runtime', title='Runtime (in hours)'),
    y=alt.Y('weekly_views', title='Weekly Views (in hours)'),
    tooltip=[
        alt.Tooltip('runtime:Q', title='Runtime (in hours)'),
        alt.Tooltip('weekly_views:Q', title="Weekly Views (in hours)", format=','),
        alt.Tooltip('show_title:N', title='Film Title'),
    ]
).properties(
    title='Runtime vs. Weekly Viewership of Netflix Films (2023-2024)',
    width=600,
    height=500
).interactive()

# View Plot
scatterplot

alt.Chart(...)

### Subquestion
**Does the runtime of the top 50 Netflix films influence their weekly viewership?**

In [5]:
# Sort values from most to least appeared in the top 10 rankings
longest_in_top_10 = films_grouped.sort_values('week', ascending=False)

# Take the top 50 films that appeared the most in the top 10 rankings
top_50 = longest_in_top_10.head(50)

# Plot
scatterplot = alt.Chart(top_50).mark_circle(size=100).encode(
    x=alt.X('runtime', title='Runtime (in hours)'),
    y=alt.Y('weekly_views', title='Weekly Views (in hours)'),
    tooltip=[
        alt.Tooltip('runtime:Q', title='Runtime (in hours)'),
        alt.Tooltip('weekly_views:Q', title="Weekly Views (in hours)", format=','),
        alt.Tooltip('show_title:N', title='Film Title'),
    ]
).properties(
    title='Runtime vs. Weekly Viewership of the Top 50 Netflix Films (2023-2024)',
    width=600,
    height=500
).interactive()

# View Plot
scatterplot

alt.Chart(...)

## Question 2

**Is viewership for Netflix films the highest during the winter compared to all other calendar seasons?**

In [6]:
# Convert the 'week' column to datetime format
films['week'] = pd.to_datetime(films['week'])

# Define a dictionary mapping months to seasons
seasons = {
    'Winter': (12,1,2), # December, January, February
    'Spring': (3,4,5),  # March, April, May
    'Summer': (6,7,8),  # June, July, August
    'Fall'  : (9,10,11) # September, October, November
}

# Function to determine the season based on the month
def get_season(month):
  for season, months in seasons.items():
    if month in months:
      return season

# Create a new column 'season' based on the month, mapping it to its corresponding season
films['season'] = films['week'].dt.month.map(get_season)

# Calculate the total weekly viewership per season
seasonal_viewership = films.groupby(['season'])['weekly_views'].sum().reset_index()

# Plotting
chart = alt.Chart(seasonal_viewership).mark_bar().encode(
    x=alt.X('season', title='Calendar Season', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('weekly_views', title='Total Viewership'),
    color=alt.condition(
        alt.datum.season=='Winter',
        alt.value('steelblue'),
        alt.value('gray')
    ),
    tooltip=[
        alt.Tooltip('season:N', title='Calendar Season'),
        alt.Tooltip('weekly_views:Q', title="Total Viewership", format=',')
    ]
).properties(
    title='Total Viewership Per Calendar Season of Netflix Films (2021-)',
    width=600,
    height=500
)

# View plot
chart

<ipython-input-6-3dff3bb77c88>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films['week'] = pd.to_datetime(films['week'])
<ipython-input-6-3dff3bb77c88>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films['season'] = films['week'].dt.month.map(get_season)


alt.Chart(...)